In [1]:
pip install py7zr

StatementMeta(, 4f5a0c6f-b96b-42c6-8dce-4b22515c1dba, 3, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 72.3 MB/s eta 0:00:0000:01
  Attempting uninstall: brotli
    Found existing installation: Brotli 1.0.9
    Uninstalling Brotli-1.0.9:
      Successfully uninstalled Brotli-1.0.9
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import py7zr
import lzma
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pandas import * 
from datetime import datetime
from notebookutils import mssparkutils
import re
import unicodedata

StatementMeta(, 4f5a0c6f-b96b-42c6-8dce-4b22515c1dba, 4, Finished, Available, Finished)

In [3]:
def descompacta(raiz,arquivo):
    with py7zr.SevenZipFile(os.path.join(raiz,arquivo),'r') as archive:
        archive.extractall(path=raiz)

StatementMeta(, 4f5a0c6f-b96b-42c6-8dce-4b22515c1dba, 5, Finished, Available, Finished)

In [4]:
def descompacta(raiz,arquivo):
    try:
        with py7zr.SevenZipFile(os.path.join(raiz,arquivo),'r') as archive:
            archive.extractall(path=raiz)
    except lzma.LZMAError as e: #capturando o erro diretamente do módulo lzma
        print(f"Erro ao descompactar{arquivo}: {e}")
    except Exception as e: #capturando outros erros possíveis
        print(f"Erro inesperado ao descompactar{arquivo}: {e}")

def listar_arquivos_7z(diretorio, palavra_excluida):
    caminhos_7z=[]
    for raiz, diretorios, arquivos in os.walk(diretorio):
        for arquivo in arquivos:
            caminho_completo=os.path.join(raiz, arquivo)
            if arquivo.endswith('.7z') and palavra_excluida not in caminho_completo:
                descompacta(raiz=raiz, arquivo=arquivo)
                print(raiz)
                caminhos_7z.append(caminho_completo)
    return caminhos_7z

StatementMeta(, 4f5a0c6f-b96b-42c6-8dce-4b22515c1dba, 6, Finished, Available, Finished)

In [18]:
caminho_diretorio='/lakehouse/default/Files/caged/'
palavra_excluida='Legado'
caminhos_7z=listar_arquivos_7z(caminho_diretorio, palavra_excluida)

StatementMeta(, 4f5a0c6f-b96b-42c6-8dce-4b22515c1dba, 20, Finished, Available, Finished)

/lakehouse/default/Files/caged/2020/202001
/lakehouse/default/Files/caged/2020/202002
Erro ao descompactarCAGEDMOV202002.7z: Corrupt input data
/lakehouse/default/Files/caged/2020/202002
/lakehouse/default/Files/caged/2020/202003
/lakehouse/default/Files/caged/2020/202003
/lakehouse/default/Files/caged/2020/202004
/lakehouse/default/Files/caged/2020/202004
/lakehouse/default/Files/caged/2020/202004
/lakehouse/default/Files/caged/2020/202005
/lakehouse/default/Files/caged/2020/202005
/lakehouse/default/Files/caged/2020/202005
/lakehouse/default/Files/caged/2020/202006
/lakehouse/default/Files/caged/2020/202006
Erro inesperado ao descompactarCAGEDMOV202006.7z: invalid header data
/lakehouse/default/Files/caged/2020/202006
/lakehouse/default/Files/caged/2020/202007
Erro ao descompactarCAGEDFOR202007.7z: Corrupt input data
/lakehouse/default/Files/caged/2020/202007
/lakehouse/default/Files/caged/2020/202007
/lakehouse/default/Files/caged/2020/202008
Erro ao descompactarCAGEDFOR202008.7z: C

In [14]:
caminho_diretorio='/lakehouse/default/Files/rais/'
palavra_excluida='Legado'
caminhos_7z=listar_arquivos_7z(caminho_diretorio, palavra_excluida)

StatementMeta(, 4f5a0c6f-b96b-42c6-8dce-4b22515c1dba, 16, Finished, Available, Finished)

/lakehouse/default/Files/rais/estabelecimento


In [19]:
df = spark.read.format('csv')\
               .option('header',True)\
               .option('delimiter',';')\
               .load("Files/caged/2*/*/*.txt")
display(df)

StatementMeta(, 4f5a0c6f-b96b-42c6-8dce-4b22515c1dba, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4aa8b0e1-6917-46b6-ab68-5489324014f8)

In [8]:
df.write.format('delta').mode("overwrite").option("overwriteSchema","true")\
        .saveAsTable('novo_caged')

StatementMeta(, 8eace06f-4a17-46b8-aeff-259904521575, 10, Finished, Available, Finished)

In [16]:
df = spark.read.format('csv')\
               .option('header',True)\
               .option('delimiter',';')\
               .load("Files/rais/estabelecimento.txt")
display(df)

StatementMeta(, 4f5a0c6f-b96b-42c6-8dce-4b22515c1dba, 18, Finished, Available, Finished)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: abfss://545a8b35-1b83-4b8a-b984-0261aa9de6ee@onelake.dfs.fabric.microsoft.com/993a8a71-9951-48ed-b7c6-591d5af28027/Files/rais/estabelecimento.txt.

In [ ]:
df.write.format('delta').mode("overwrite").option("overwriteSchema","true")\
        .saveAsTable('rais')